### Slitless Spectroscopy Example - JWST NIRISS Grism

This notebook shows exposure level processing of JWST NIRISS Grism observations. It focuses on explaining the new WCS tools made available for JWST.

Essentially the direct image acts as a reference point in the WCS pipeline with its transform to sky applied in one part of the WCS pipeline and a transform from direct to grism image appended to it to form the complete WCS pipeline from sky to grism image.

`sky` (RA, DEC, WAVE, ORDER) --> `direct_image` (IMX, IMY, WAVE, ORDER) --> `grism_image` (GRISMX, GRISMY) 

#### Reference files 

- The transform from `sky` to `direct_image` uses a reference file with distortion model a nd a reference file where filter offsets are listed.

- The transform from direct image to grism image is a polynomial which defines the spectral trace on the grism image. 

- A reference file containing the wavelength range for every spectral order 

All files are in ASDF format.

**Note:**
The following two variables need to be set in the terminal in order to access reference files

export CRDS_PATH=<locally-accessable-path>/crds_cache/jwst_ops
    
export CRDS_SERVER_URL=https://jwst-crds.stsci.edu

#### Processing workflow

- The direct images (one or more dither observations) are processed through the Level 2 Imaging Pipeline and then the Level3 Imaging pipepine. 
- A source detection algorithm is run on the combined resampled image and a source catalog is produced. 
- The grism image is run through the Level 2 spectral pipeline using the source catalog produced by processing the direct images.
 - The assign_wcs step constructs a WCS pipeline which passes through the following coordinate frames
 
 `grism_image -> direct_image -> V2V3 -> world`
 
            `specwcs`     `distortion`
            
 - The extract_2d step makes cutouts of each slit and saves them as separate arrays with their own WCS. The bounding box of a spectrum on the detector is determined by the extent of the object on the sky, determined by the segmentation map and transformed to grism coordinates using the inverse WCS transform.
                              

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import astropy
from astropy.io import fits

import numpy as np
import asdf
#pipeline
import jwst
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline
#wcs
from jwst import assign_wcs
from jwst.assign_wcs import assign_wcs_step
from jwst.assign_wcs import AssignWcsStep
from jwst.extract_2d import Extract2dStep
#datamodels
from jwst import datamodels

##### Run Level2 and Level 3 of the Imaging Pipeline

Level 2 processing runs the `assign_wcs` step which generates the imaging WCS pipeline and attaches it to the science observation.

Level 3 processing generates a source catalog.

In [2]:
fimage = '../data/jw01448013001_02106_00001_nis_rate.fits'
outim2 = Image2Pipeline.call(fimage)

2023-06-02 10:18:11,969 - CRDS - INFO -  Fetching  /Users/dencheva/crds/mappings/jwst/jwst_system_calver_0033.rmap    4.1 K bytes  (1 / 5 files) (0 / 13.6 K bytes)
2023-06-02 10:18:12,075 - CRDS - INFO -  Fetching  /Users/dencheva/crds/mappings/jwst/jwst_system_0032.imap      385 bytes  (2 / 5 files) (4.1 K / 13.6 K bytes)
2023-06-02 10:18:12,171 - CRDS - INFO -  Fetching  /Users/dencheva/crds/mappings/jwst/jwst_nircam_photom_0021.rmap    3.4 K bytes  (3 / 5 files) (4.5 K / 13.6 K bytes)
2023-06-02 10:18:12,274 - CRDS - INFO -  Fetching  /Users/dencheva/crds/mappings/jwst/jwst_nircam_0259.imap    5.1 K bytes  (4 / 5 files) (7.9 K / 13.6 K bytes)
2023-06-02 10:18:12,390 - CRDS - INFO -  Fetching  /Users/dencheva/crds/mappings/jwst/jwst_1090.pmap            580 bytes  (5 / 5 files) (13.0 K / 13.6 K bytes)
2023-06-02 10:18:12,643 - stpipe - INFO - PARS-IMAGE2PIPELINE parameters found: /Users/dencheva/crds/references/jwst/niriss/jwst_niriss_pars-image2pipeline_0002.asdf
2023-06-02 10:18:12

2023-06-02 10:18:13,485 - stpipe.Image2Pipeline.assign_wcs - INFO - assign_wcs updated S_REGION to POLYGON ICRS  215.778070940 53.224609849 215.785525030 53.187469098 215.847028700 53.192061303 215.839648019 53.229180841
2023-06-02 10:18:13,486 - stpipe.Image2Pipeline.assign_wcs - INFO - COMPLETED assign_wcs
2023-06-02 10:18:13,562 - stpipe.Image2Pipeline.assign_wcs - INFO - Step assign_wcs done
2023-06-02 10:18:13,643 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field running with args (<ImageModel(2048, 2048) from jw01448013001_02106_00001_nis_rate.fits>,).
2023-06-02 10:18:13,644 - stpipe.Image2Pipeline.flat_field - INFO - Step flat_field parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '../data', 'save_interpolated_flat': False, 'user_supplied_flat': None, 'inve

In [3]:
Image3Pipeline.from_cmdline(["calwebb_image3", "jw01448013001_02106_00001_nis_cal.fits"])

2023-06-02 10:18:15,530 - stpipe - INFO - PARS-TWEAKREGSTEP parameters found: /Users/dencheva/crds/references/jwst/niriss/jwst_niriss_pars-tweakregstep_0001.asdf
2023-06-02 10:18:15,548 - stpipe - INFO - PARS-OUTLIERDETECTIONSTEP parameters found: /Users/dencheva/crds/references/jwst/niriss/jwst_niriss_pars-outlierdetectionstep_0002.asdf
2023-06-02 10:18:15,577 - CRDS - ERROR -  Error determining best reference for 'pars-sourcecatalogstep'  =   No match found.
2023-06-02 10:18:15,587 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2023-06-02 10:18:15,589 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2023-06-02 10:18:15,592 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2023-06-02 10:18:15,595 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2023-06-02 10:18:15,597 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-06-02 10:18:15,600 - stpipe.

2023-06-02 10:18:28,069 - stpipe.Image3Pipeline.resample - INFO - Update S_REGION to POLYGON ICRS  215.778022554 53.224774902 215.785500441 53.187470651 215.847069976 53.191881709 215.839644948 53.229189802
2023-06-02 10:18:28,576 - stpipe.Image3Pipeline.resample - INFO - Saved model in jw01448013001_02106_00001_nis_i2d.fits
2023-06-02 10:18:28,577 - stpipe.Image3Pipeline.resample - INFO - Step resample done
2023-06-02 10:18:28,657 - stpipe.Image3Pipeline.source_catalog - INFO - Step source_catalog running with args (<ImageModel(2063, 2040) from jw01448013001_02106_00001_nis_i2d.fits>,).
2023-06-02 10:18:28,659 - stpipe.Image3Pipeline.source_catalog - INFO - Step source_catalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'cat', 'search_output_file': True, 'input_dir': '', 'bkg_boxsize': 1000, 'kernel_fwhm': 2.0, 'snr

Level 3 processing produces a catalog - jw01448013001_02106_00001_nis_cat.ecsv.

The output image from L3 pipeline is jw01448013001_02106_00001_nis_i2d.fits.

##### assign_wcs
Run the assign_wcs step on the spectral image. It uses the reference files to chain all transforms and generate a WCS pipeline

In [4]:
# Process one grism observation using the new reference file
fname = "../data/jw01448013001_02105_00001_nis_rate.fits"
astep = AssignWcsStep()
out = astep.call(fname)

2023-06-02 10:18:32,558 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2023-06-02 10:18:32,949 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2023-06-02 10:18:33,046 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args ('../data/jw01448013001_02105_00001_nis_rate.fits',).
2023-06-02 10:18:33,048 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.1, 'sip_degree': None, 'sip_max_inv_pix_error': 0.1, 'sip_inv_degree': None, 'sip_npoints': 12, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2023-06-02 10:18:33,806 - stpipe.AssignWcsStep - INFO - Added Barycentric velocity correction: 0.999965786033709
2023-06-02 10:18:34,038 - stpipe.AssignW

##### Extract_2d

Run the extract_2d step in the pipeline. It uses the source catalog and the WCS object to find the location of the trace for each slit and order and extract a cutout with the corresponding WCS. The limits of the cutpout are defined from the segmentation map of the image on the sky. The corners on the sky are transformed to the grism detector using the inverse WCS transform.

In [5]:
out.meta.source_catalog="jw01448013001_02106_00001_nis_cat.ecsv"
exout = Extract2dStep.call(out, wfss_mmag_extract=21)
exout.save("jw01448013001_02105_00001_nis_ex2d.fits")

2023-06-02 10:18:34,366 - stpipe.Extract2dStep - INFO - Extract2dStep instance created.
2023-06-02 10:18:34,468 - stpipe.Extract2dStep - INFO - Step Extract2dStep running with args (<ImageModel(2048, 2048) from jw01448013001_02105_00001_nis_rate.fits>,).
2023-06-02 10:18:34,470 - stpipe.Extract2dStep - INFO - Step Extract2dStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_name': None, 'extract_orders': None, 'grism_objects': None, 'tsgrism_extract_height': None, 'wfss_extract_half_height': 5, 'wfss_mmag_extract': 21, 'wfss_nbright': 1000}
2023-06-02 10:18:34,493 - stpipe.Extract2dStep - INFO - EXP_TYPE is NIS_WFSS
2023-06-02 10:18:34,522 - stpipe.Extract2dStep - INFO - Extracting objects < abmag = 21
2023-06-02 10:18:34,522 - stpipe.Extract2dStep - INFO - Getti

2023-06-02 10:18:50,101 - stpipe.Extract2dStep - INFO - Finished extractions
2023-06-02 10:18:50,117 - stpipe.Extract2dStep - INFO - Results used CRDS context: jwst_1090.pmap
2023-06-02 10:18:50,118 - stpipe.Extract2dStep - INFO - Step Extract2dStep done


'jw01448013001_02105_00001_nis_ex2d.fits'

In [6]:
exout.info()

root (AsdfObject)
├─meta (dict)
│ ├─model_type (str): MultiSlitModel
│ ├─date (str): 2023-06-02T14:18:35.410
│ ├─origin (str): STSCI
│ ├─time_sys (str): UTC
│ ├─time_unit (str): s
│ ├─filename (str): jw01448013001_02105_00001_nis_ex2d.fits
│ ├─data_processing_software_version (str): 2022_1b
│ ├─prd_software_version (str): PRDOPSSOC-045-002
│ ├─oss_software_version (str): 8.4.2
│ ├─calibration_software_version (str): 1.10.2
│ ├─calibration_software_revision (str): RELEASE
│ ├─telescope (str): JWST
│ ├─hga_move (bool): False
│ ├─pwfseet (float): 59704.74025150463
│ ├─nwfsest (float): 0.0
│ ├─program (dict) ...
│ ├─observation (dict) ...
│ ├─visit (dict) ...
│ ├─target (dict) ...
│ └─17 not shown
└─slits (list) ...
Some nodes not shown.


In [7]:
print(len(exout.slits))

30


##### Plots

Plot the grism image with the bounding_boxes (cutouts) of the individual slits and the location of the sources.

In [8]:
ax = plt.subplot()
ax.imshow(out.data, aspect='auto', vmin=-1, vmax=1.4, origin='lower')
plt.gray()

for slit in exout.slits:
    ax.scatter(slit.source_xpos, slit.source_ypos, c='r')
    bbox = slit.meta.wcs.bounding_box
    xstart, ystart = slit.xstart, slit.ystart
    ax.add_patch(patches.Rectangle((bbox[0][0]+xstart-1, bbox[1][0]+ystart-1), 
                                   bbox[0][1]-bbox[0][0], bbox[1][1]-bbox[1][0], edgecolor='g',
                                   fill=False))

plt.gray()

<IPython.core.display.Javascript object>

2023-06-02 10:18:54,590 - stpipe - WARNING - /Users/dencheva/opt/anaconda3/envs/b92/lib/python3.9/site-packages/ipykernel/comm/comm.py:79: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  warn(



In [9]:
specwcs = "../data/jwst_niriss_specwcs_0018.asdf"

distortion = "../data/jwst_niriss_distortion_0013.asdf"

filteroffset = "jwst_niriss_filteroffset_0002.asdf"

wavelength_range = "../data/jwst_niriss_wavelengthrange_0002.asdf"


In [10]:
aspecwcs = asdf.open(specwcs)
asdf.info(specwcs, max_rows=200, max_cols=200)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 2.11.1
├─history (dict)
│ ├─entries (list)
│ │ └─[0] (HistoryEntry)
│ │   ├─description (str): NIRISS Grism Parameters
│ │   ├─software (Software)
│ │   │ ├─author (str): Swara Ravindranath
│ │   │ ├─homepage (str): https://github.com/spacetelescope/jwreftools
│ │   │ ├─name (str): niriss_reftools.py
│ │   │ └─version (str): 0.7.1
│ │   └─time (datetime)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension.BuiltinExtension
│   │ └─software (Software)
│   │   ├─name (str): asdf
│   │   └─version (str): 2.11.1
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.5.0
│   │ └─software (Software)
│   │   ├─name (str): asdf-astropy
│   │   └─version 

In [11]:
asdf.info(distortion, max_rows=200, max_cols=200)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 2.8.3
├─history (dict)
│ ├─entries (list)
│ │ └─[0] (HistoryEntry)
│ │   ├─description (str): This file is being delivered because it is the wcs reference and distortion correction reference file. Documentation and scripts can be found in  (truncated)
│ │   └─time (datetime)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension.BuiltinExtension
│   │ └─software (Software)
│   │   ├─name (str): asdf
│   │   └─version (str): 2.8.3
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/transform/extensions/transform-1.5.0
│   │ └─software (Software)
│   │   ├─name (str): asdf-astropy
│   │   └─version (str): 0.1.2
│   └─[2] (ExtensionMetadata)
│     ├─extension_class (str)

In [12]:
asdf.info(wavelength_range, max_rows=500, max_cols=500)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): Space Telescope Science Institute
│ ├─homepage (str): http://github.com/spacetelescope/asdf
│ ├─name (str): asdf
│ └─version (str): 2.1.0.dev1417
├─history (dict)
│ ├─entries (list)
│ │ └─[0] (HistoryEntry)
│ │   ├─description (str): Ground NIRCAM wavelength ranges, updated to include specific extraction orders per filter and a clearer structure for the wavelengthrange property
│ │   ├─software (Software)
│ │   │ ├─author (str): STScI
│ │   │ ├─homepage (str): https://github.com/spacetelescope/jwreftools
│ │   │ ├─name (str): niriss_reftools.py
│ │   │ └─version (str): 0.7.1
│ │   └─time (datetime)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension.BuiltinExtension
│   │ └─software (dict)
│   │   ├─name (str): asdf
│   │   └─version (str): 2.1.0.dev1417
│   └─[1] (ExtensionMetadata)
│     ├─extension_class (str): astropy.io.misc.asdf.extension.AstropyAsdfExtension
│     └─softwa